This notbook is used to split the ingredients into multiple columns for easy analysis.

In [26]:
# import dependencies
import pandas as pd
import numpy as np
import re
from re import search

In [27]:
# import the raw csv
recipes_df_raw = pd.read_csv('technical_list.csv', encoding="utf-8")
recipes_df = recipes_df_raw.copy()
recipes_df_raw.head

<bound method NDFrame.head of                                                  title           judge  \
0                   Rav Gill’s Macaron Snack-Cessories    Ravneet Gill   
1    Liam Charles’s Biscuit Card Tower & Deck of Cards    Liam Charles   
2                 Rav Gill’s Cherry & Almond Deco Roll    Ravneet Gill   
3               Liam Charles’s Yorkshire Pudding Wraps    Liam Charles   
4          Rav Gill’s Churros with Banana Butterscotch    Ravneet Gill   
..                                                 ...             ...   
124                    Mary Berry’s Hazelnut Dacquoise  Paul Hollywood   
125                           Mary Berry’s Religieuses  Paul Hollywood   
126                  Paul Hollywood’s Apricot Couronne  Paul Hollywood   
127          Mary Berry’s Tuiles with Chocolate Mousse  Paul Hollywood   
128                      Mary Berry’s Charlotte Royale  Paul Hollywood   

            makes              difficulty hands_on_time baking_time  \
0         

In [28]:
# # Make a function to remove p tags from the list
# def remove_tags(ingredients):
#         clean_ingredients = ingredients.replace('<p>','').replace('</p>','')
#         return clean_ingredients

# # iterate through each ingredient list and remove p tags.
# # no longer needed
# ingredients = recipes_df_raw['ingredients']
# recipes_no_tags = recipes_df_raw.copy()
# recipes_no_tags['ingredients_clean'] = ''
# for i in range(len(ingredients)):
#     ingredient_list = ingredients[i]
#     clean_list = remove_tags(ingredient_list)
#     recipes_no_tags['ingredients_clean'][i] = clean_list
# print(recipes_no_tags['ingredients_clean'])

In [29]:
# Formats for ingredients
# 1g unsalted butter
# 1 tsp vanilla extract
# slivered pistachios
# 2 large eggs
# 2 dill pickles, quartered lengthways
# juice of 1/2 lemon

def split_ingredient(ingredient):
    '''
    The split_ingredients funcion takes an ingredient list with each ingredient inside a <p></p> tag. The 
    '''
    comment = 'na'
    quantity = 'na'
    measurement = 'na'
    item = 'na'

    # if the ingredient contains a comment (need to deal when the comment is before the recipe, like in 50)
    if ',' in ingredient:
        comment = ingredient.split(',')[1]
        ingredient = ingredient.split(',')[0]

    # if ingredient is eggs
    if 'egg' in ingredient:
        quantity = re.findall('\d+', ingredient)
        measurement = re.findall('\d+(.*) ', ingredient)
        return 'it has eggs yo'
    #if the ingredient contains no numbers


    # if measurement is weight or volume
    elif re.match('^\d+[a-zA-Z]',ingredient) is not None:
        quantity = re.findall('\d+', ingredient)[0]
        measurement = re.findall('^\d+([a-zA-Z]+) ', ingredient)[0]
        item = re.findall('\d+[a-zA-Z]+ (.*)', ingredient)[0]

    # Juice of (still need zest of)
    elif 'juice of' in ingredient:
        item = re.findall('juice of [^a-z]+([a-z ]+)', ingredient, re.IGNORECASE)[0]
        if 'zest' in ingredient:
            comment = 'juice and zest'
        else:
            comment = 'juice'
        if re.findall('[0-9]+', ingredient) is not None:
            quantity = re.findall('[0-9]+', ingredient)[0]
        else:
            quantity = '1'


    # if ingredient uses tsp or tbsp
    elif 'tbsp' in ingredient: 
        quantity = ingredient.split(' ',1)[0]
        measurement = 'tbsp'
        item = re.findall('tbsp (.*)', ingredient)[0]

    elif 'tsp' in ingredient:
        quantity = ingredient.split(' ')[0]
        measurement = 'tsp'
        item = re.findall('tsp (.*)', ingredient)[0]
    
    # if it just a number and ingredient (2 pickles)
    elif re.match('^[0-9]+/*[0-9]* ', ingredient) is not None:
        quantity = re.findall('^([0-9]+/*[0-9]*)', ingredient)[0]
        item = re.findall('^[0-9]+/*[0-9]* (.*)', ingredient)[0]
    
    # a pinch of something
    elif search('pinch', ingredient) is not None:
        measurement = re.findall('(.*pinch)', ingredient)[0]
        item = re.findall('.*pinch of (.*)', ingredient)[0]
        if re.match('^[0-9]', ingredient) is not None:
            quantity = re.findall('^([0-9])', ingredient)[0]
    


    # if if ingredient is juice of
    else:
        return ingredient
    
    ingredient_list_split = {'quantity':quantity, 'measurement':measurement, 'ingredient':item, 'comment':comment}
    return ingredient_list_split

In [30]:
# still need to deal with line 22 in the csv with the <br>
def seperate_ingredients(ingredient_string):
    '''
    The seperate_ingredients funcion takes an ingredient list with each ingredient inside a <p></p> tag and returns a list containing all the ingredients
    '''
    ingredient_list = re.findall('<p>(.*?)</p>', ingredient_string)
    return ingredient_list

In [31]:
test_ingredient = '200ml whole milk'
print(split_ingredient(test_ingredient))

{'quantity': '200', 'measurement': 'ml', 'ingredient': 'whole milk', 'comment': 'na'}


In [32]:
def get_ingredients():
    '''
    
    '''

In [33]:
recipes_df['ing_split'] = ""
for i in range(recipes_df.size):
    ingredient_list = seperate_ingredients(recipes_df['ingredients'][i])
    ingredients = []
    for ingredient in ingredient_list:
        ingredients.append(split_ingredient(ingredient))
    
    recipes_df['ing_split'][i] = ingredients

IndexError: list index out of range

In [17]:
ingredient_list = seperate_ingredients(recipes_df['ingredients'][20])
for i in ingredient_list:
    print(split_ingredient(i))

{'quantity': '400', 'measurement': 'ml', 'ingredient': 'whole milk ', 'comment': 'na'}
{'quantity': '45', 'measurement': 'g', 'ingredient': 'caster sugar ', 'comment': 'na'}
it has eggs yo
{'quantity': '25', 'measurement': 'g', 'ingredient': 'unsalted butter', 'comment': ' melted '}
{'quantity': '125', 'measurement': 'g', 'ingredient': 'plain flour', 'comment': 'na'}
{'quantity': '15', 'measurement': 'g', 'ingredient': 'cornflour', 'comment': 'na'}
{'quantity': '½', 'measurement': 'tsp', 'ingredient': 'baking powder', 'comment': 'na'}
{'quantity': '15', 'measurement': 'g', 'ingredient': 'matcha powder', 'comment': ' plus extra for dusting '}
Sunflower oil


In [18]:
recipes_df['ingredients'][23]

'[<p>375g strong white bread flour </p>, <p>5g fast-action dried yeast</p>, <p>2 tsp caster sugar</p>, <p>1 tsp fine salt</p>, <p>240ml lukewarm water </p>, <p>Red, orange, yellow, green and blue food-colouring paste</p>, <p>1 tsp bicarbonate of soda </p>]'